# Extract

In [34]:
# Make jupyter to accept import python modules from other folders
import sys
sys.path.append("../..")

from utils.functions import DB_MinioClient
from utils.constants import Constants
import pandas as pd
import io
import json

FILE_NAME = "pistacho.json"
BUCKET_FROM_NAME = Constants.STORAGE_LANDING_ZONE.value

minio_client = DB_MinioClient().connect()
data = minio_client.get_object(
    BUCKET_FROM_NAME, FILE_NAME).read().decode("utf-8")
data_json = json.loads(data)

# Validate

In [35]:
df_enclosures = pd.json_normalize(data_json["Exploitation Parcel"])

import pandera as pa
# Validate with pandera schema
schema = {
    "Parcel_Id": pa.Column(pa.String),
    "CAP_Code.Province.Province_Code": pa.Column(pa.String),
    "CAP_Code.Province.Province_Name": pa.Column(pa.String, nullable=True),
    "CAP_Code.Municipality.Municipality_Code": pa.Column(pa.String),
    "CAP_Code.Municipality.Municipality_Name": pa.Column(pa.String, nullable=True),
    "CAP_Code.Agregate": pa.Column(pa.String),
    "CAP_Code.Zone": pa.Column(pa.String),
    "CAP_Code.Polygon": pa.Column(pa.String),
    "CAP_Code.Parcel": pa.Column(pa.String),
    "CAP_Code.Enclosure": pa.Column(pa.String),
    "Use": pa.Column(pa.String),
    "Official_Area": pa.Column(pa.Float),
    "Used_Area": pa.Column(pa.Float),
    "Crop.Species": pa.Column(pa.String),
    "Crop.Variety": pa.Column(pa.String),
    "Rainfed_Irrigated": pa.Column(pa.String),
    "Protected_Area?": pa.Column(pa.String),
}

schema = pa.DataFrameSchema(schema, coerce=True)
df_enclosures = schema.validate(df_enclosures)
df_enclosures.head()
    

,Parcel_Id,Use,Official_Area,Used_Area,Rainfed_Irrigated,Protected_Area?,CAP_Code.Province.Province_Code,CAP_Code.Province.Province_Name,CAP_Code.Municipality.Municipality_Code,CAP_Code.Municipality.Municipality_Name,CAP_Code.Agregate,CAP_Code.Zone,CAP_Code.Polygon,CAP_Code.Parcel,CAP_Code.Enclosure,Crop.Species,Crop.Variety
0,29,FS,13.63,13.63,Irrigated,N,47,,96,MORALEJA DE LAS PANADERAS,,,5,20,1,PISTACHO,KERMAN
1,27,TA,0.73,0.73,Irrigated,N,47,,124,POZAL DE GALLINAS,,,5,5170,2,PISTACHO,KERMAN
2,27,FS,2.14,2.14,Irrigated,N,47,,124,POZAL DE GALLINAS,,,4,560,1,PISTACHO,KERMAN
3,26,VI,1.46,1.46,Irrigated,N,47,,124,POZAL DE GALLINAS,,,5,585,2,PISTACHO,KERMAN
4,26,FS,3.40,3.40,Irrigated,N,47,,124,POZAL DE GALLINAS,,,5,5221,1,PISTACHO,KERMAN


# Clean

## Enclosures

In [36]:


# If municipality, province, agregate, zone, polygon, parcel and enclosure are "", then set them to 0
possible_zero_columns = ["CAP_Code.Municipality.Municipality_Code", "CAP_Code.Province.Province_Code", "CAP_Code.Agregate", "CAP_Code.Zone", "CAP_Code.Polygon", "CAP_Code.Parcel", "CAP_Code.Enclosure"]
for column in possible_zero_columns:
    if df_enclosures[column].dtype == "object":
        df_enclosures[column] = df_enclosures[column].replace(["", " "], 0)

df_enclosures

# Rename columns
df_enclosures = df_enclosures.rename(columns={
    "CAP_Code.Province.Province_Code": "parcelProvinceId",
    "CAP_Code.Municipality.Municipality_Code": "parcelMunicipalityId",
    "CAP_Code.Polygon": "parcelPolygonId",
    "CAP_Code.Parcel": "parcelId",
    "CAP_Code.Enclosure": "parcelEnclosureId",
    "CAP_Code.Municipality.Municipality_Name": "parcelGeographicSpot",
    "CAP_Code.Agregate": "parcelAggregatedId",
    "CAP_Code.Zone": "parcelZoneId",
    "Use": "parcelUse",
    "Official_Area": "areaSIGPAC",
    "Used_Area": "area",
    "Crop.Species": "cropName",
    "Crop.Variety": "parcelVarietyId",
    "Rainfed_Irrigated": "irrigationKind",
    "Protected_Area?": "protectedArea",
})


# Drop columns
df_enclosures.drop(columns=["Parcel_Id"], inplace=True)

# Add columns
df_enclosures["cropId"] = "445"
df_enclosures["tenureRegimeId"] = None
df_enclosures["plantationYear"] = None
df_enclosures["numberOfTrees"] = None
df_enclosures["plantationDensity"] = None
df_enclosures["ATRIA_ADV_ASV"] = None
df_enclosures["parcelVulnerableArea"] = None
df_enclosures["specificZones"] = None
df_enclosures["slope"] = 0
df_enclosures["UHC"] = None
df_enclosures["UHCDescription"] = None
df_enclosures["ZepaZone"] = None
df_enclosures["SIEZone"] = None
# # Convert NULL, NP, NaN, etc. to None
# df_enclosures = df_enclosures.replace(
#     {pd.NA: None, "NP": None, "NaN": None, "": None, "NULL": None})
# df_enclosures

## Treatments

In [37]:
# df_treatments = pd.json_normalize(data_json["Phytosanitary_Action"])

# # Map Parcel_Id from df_enclosures to df_treatments
# df_treatments["enclosureId"] = df_treatments["Parcel_Id"].apply(
#     lambda x: df_enclosures.loc[df_enclosures["Parcel_Id"] == x, "enclosureId"].values[0])
# df_treatments

# Transform

In [38]:
# Transform df_enclosures to parquet
parquet_enclosures = df_enclosures.to_parquet()

# Load

In [39]:
BUCKET_TO_NAME = Constants.STORAGE_TRUSTED_ZONE.value
# Create bucket if it doesn't exist
if not minio_client.bucket_exists(BUCKET_TO_NAME):
    minio_client.make_bucket(BUCKET_TO_NAME)

minio_client.put_object(
    BUCKET_TO_NAME,
    "ERP/PISTACYL/2021/pistachio_enclosures_2021.parquet",
    io.BytesIO(parquet_enclosures),
    length=io.BytesIO(parquet_enclosures).getbuffer().nbytes,
    content_type="application/octet-stream",
    metadata={
        "type": Constants.METADATA_PARCELS_AND_TREATMENTS_PARCELS.value,
        "source": "PISTACYL",
        "year": 2021,
        "state": "processed",
    }
)